In [1]:
# System variables
import os 
from glob import glob

# Data processing libraries
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

# Dataset connection
import opendatasets as od

# Graphic tools 
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Download dataset
od.download("https://www.kaggle.com/datasets/yasirabdaali/artificial-intelligence-journals-ranking-20002021")

In [ ]:
def path_csvFiles (PATH = os.getcwd(),EXT = "*.csv"):  
    """
    Retrieve all files with a given extension (EXT = *.csv by default) from current
    working directory where the process is being implemented, using os.getcwd () 
    and the glob module, which finds all path names that resemble a specified pattern 
    according to the rules that are followed in a Unix terminal.
    
    Returns:
        Returns a list with all the files path of the given extension 
    """
    list_paths = []
    for path, subdir, files in os.walk(PATH):
        for file in glob(os.path.join(path, EXT)):
            if file.find("scimagojr") != -1:
                list_paths.append(file)
    return list_paths


def concat_paths(all_paths):
    """receives a list of file directories with a CSV extension 
    and creates a dataset by concatenating each file
    and assigning each file a year label that is stored in the "Year" column,
    returning a dataframe

    Returns: 
        Returns a dataframe
    """

    all_df = []
    for path in all_paths:
        df = pd.read_csv(path, sep = ';')        
        df['Year'] = int(path.split()[1])
        all_df.append(df)
    
    df = pd.concat(all_df, ignore_index=True)
    return df 

In [ ]:
df = concat_paths(path_csvFiles())
df.head(5)

In [ ]:
#Visualización de las columnas que representan vacios para modificarlas
sns.heatmap(df.notnull())

In [ ]:
# =============================================================================
# Plot rectangular df as a color-encoded matrix. 
# =============================================================================
sns.heatmap(df[(df['Type'] == "conference and proceedings")][["Type","Coverage","Publisher"]].notnull())

In [ ]:
#Preprocessing data

#eliminar registros conference
df = df.loc[(df['Type'] == 'journal') | (df['Type'] == 'book series')]

In [ ]:
#se almacenan los datos de las columnas Total Docs.20## en la columna Total Docs. per year y se eliminan sus duplicados
df['Total Docs. per Year'] = df[list(df.filter(regex  = '20'))].fillna('').astype(str).apply(lambda x: "".join(x), axis =1)
df['Total Docs. per Year'] = df['Total Docs. per Year'].astype(float)
df.drop(list(df.filter(regex  = '20')), inplace = True, axis=1)

In [ ]:
# =============================================================================
# Explore the data type columns and identify an assignment error
# =============================================================================
df.dtypes

In [ ]:
#Explore the values with the wrong mapping in detail and create a modification scheme
for i in (5, 12, 13):
    print(f"\033[1m {df.columns[i]}:\n\033[0m {list((df[df.columns[i]]))[:30]}\n")

In [ ]:
#Modification scheme
df['SJR'] = (df['SJR'].replace(',','.', regex=True).astype(float)).fillna(0)
df['Cites / Doc. (2years)'] = (df['Cites / Doc. (2years)'].replace(',','.', regex=True).astype(float))
df['Ref. / Doc.'] = (df['Ref. / Doc.'].replace(',','.', regex=True).astype(float))

In [ ]:
# =============================================================================
# Plot rectangular df as a color-encoded matrix. 
# =============================================================================
sns.heatmap(df.notnull())

In [ ]:
# =============================================================================
# Numeric columns
# =============================================================================
int_df = df.select_dtypes(include=['int64']).copy()
print(f"[{len(int_df)} rows x {len(int_df.columns)} columns]")

In [ ]:
# =============================================================================
# summarize the central tendency, dispersion and shape of a dataset’s distribution.
# =============================================================================
df

In [ ]:
# =============================================================================
# Categorical columns
# =============================================================================
obj_df = df.select_dtypes(include=['object']).copy()
print(f"[{len(obj_df)} rows x {len(obj_df.columns)} columns]\n")
#Categorical description
obj_df.describe()


In [ ]:
# =============================================================================
# Categorical columns sets
# =============================================================================
i=0
while i<len(obj_df.columns):    
    print(("\033[1m {}: \n \033[0m {}\n").format(obj_df.columns[i],list(set(obj_df[obj_df.columns[i]]))[:20]))
    i+=1

In [ ]:
#download dataframe consolidado 
df.to_csv('journalAI.csv',sep = ";", index=False)

In [ ]:
#generates widget profile reports from a pandas
profile = ProfileReport(df, title="Top Gun Lab Profiling Report")
profile.to_widgets()

In [ ]:
#generate a HTML report file
profile.to_file("TopGunLab_Report.html")